## Import & 전역변수

In [13]:
import requests
import json
import pickle
import os
import pandas as pd
TOT_CNT = 157116
MAX_PER_PAGE = 100
ITER_NUM = TOT_CNT//MAX_PER_PAGE+1
CHECK_POINT = 300
PEOPLE_LIST_PATH = "../people/people.pickle"
VISIT_PATH = "./visit.pickle"
NUM_DIRECTORS, NUM_ACTORS, NUM_ETC = 1000, 1000, 1000

## 체크리스트 만들기 (VISIT)



In [12]:
with open(PEOPLE_LIST_PATH, 'rb') as f :
    PEOPLE_LIST = pickle.load(f)
VISIT = {people_code : False for people_code in PEOPLE_LIST}
with open(VISIT_PATH, 'wb') as f :
    pickle.dump(VISIT, f)

## 영화진흥위원회 API를 사용하여 영화정보 가져오기

In [12]:
with open(PEOPLE_LIST_PATH, 'rb') as f :
    ORI_PEOPLE_LIST = pickle.load(f)
with open(VISIT_PATH, 'rb') as f :
    VISIT = pickle.load(f)
URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
PEOPLE_LIST = {people_code : ORI_PEOPLE_LIST[people_code]['repRoleNm'] for people_code in ORI_PEOPLE_LIST if VISIT[people_code] == False}
DIR_CNT, ACT_CNT, ETC_CNT = 0, 0, 0
idx = 0
for PEOPLE in PEOPLE_LIST :
    if DIR_CNT == NUM_DIRECTORS and ACT_CNT == NUM_ACTORS and ETC_CNT == NUM_ETC :
        break
    if PEOPLE_LIST[PEOPLE] == '감독' and DIR_CNT < NUM_DIRECTORS :
        DIR_CNT += 1
    elif PEOPLE_LIST[PEOPLE] == '배우' and ACT_CNT < NUM_ACTORS :
        ACT_CNT += 1
    elif PEOPLE_LIST[PEOPLE] != '감독' and PEOPLE_LIST[PEOPLE] != '배우' and ETC_CNT < NUM_ETC :
        ETC_CNT += 1
    else :
        continue
    VISIT[PEOPLE] = True

    if idx!=0 and idx % CHECK_POINT == 0 :
        print(f"Now : {idx}")
    params = {
        # "key" : "74c13a6087207fc051ab22dd0752be12",
       "key" : "6e5d624a2e3f9085996905e406f5c882",
        "peopleCd" : PEOPLE,
    }
    response = requests.get(URL, params=params)
    # print(response.json())
    # if idx == 10 :
    #     break
    with open(f'./json/{PEOPLE}.json', 'w') as f :
        json.dump(response.json(), f)

    idx += 1

print(f"DIR_CNT : {DIR_CNT}, ACT_CNT : {ACT_CNT}, ETC_CNT : {ETC_CNT}")

with open(VISIT_PATH, 'wb') as f :
    pickle.dump(VISIT, f)

Now : 300
Now : 600
Now : 900
Now : 1200
Now : 1500
Now : 1800
Now : 2100
Now : 2400
Now : 2700
DIR_CNT : 1000, ACT_CNT : 1000, ETC_CNT : 1000


## API를 이용해 저장한 데이터 불러오기 & 컬럼 분석

In [14]:
EACH_FILE_PATH = "./json/"
files = [file for file in os.listdir(EACH_FILE_PATH) if file.endswith('.json')]
result = {}

NUM_LEN_ONE_COL = set()
NUM_LEN_OTHER_COL = set()

for file in files :
    with open(EACH_FILE_PATH+file, 'r') as f :
        load_result = json.load(f)
        peopleInfoResult = load_result['peopleInfoResult']
    INFOS = peopleInfoResult['peopleInfo']
    people_code = INFOS['peopleCd']
    result[people_code] = INFOS
    for key, val in INFOS.items() :
        if type(val) != list :
            NUM_LEN_ONE_COL.add(key)
        else :
            NUM_LEN_OTHER_COL.add(key)

print(f"len(ONE_COL) : {len(NUM_LEN_ONE_COL)}, ONE_COL : {NUM_LEN_ONE_COL}")
print(f"len(OTHER_COL) : {len(NUM_LEN_OTHER_COL)}, OTHER_COL : {NUM_LEN_OTHER_COL}")

len(ONE_COL) : 5, ONE_COL : {'peopleNm', 'peopleCd', 'peopleNmEn', 'repRoleNm', 'sex'}
len(OTHER_COL) : 2, OTHER_COL : {'filmos', 'homepages'}


## Table 만들기

In [15]:
EACH_FILE_PATH = "./json/"
files = [file for file in os.listdir(EACH_FILE_PATH) if file.endswith('.json')]
result = {}
result_filmo = []
for idx,file in enumerate(files) :
    if idx != 0 and idx % CHECK_POINT == 0 :
        print(f"Now : {idx}")
    with open(EACH_FILE_PATH+file, 'r') as f :
        content = json.load(f)
        peopleInfoResult = content['peopleInfoResult']
        people_info = peopleInfoResult['peopleInfo']
    people_code = None
    for key, val in people_info.items() :
        if type(val) == list :
            if key == 'filmos' :
                for fil in val :
                    fil['peopleCd'] = people_code
                    result_filmo.append(fil)
            else :
                continue
        if key == 'peopleCd' :
            result[val] = dict()
            people_code = val
        try :
            result[people_code][key] = val
        except KeyError :
            continue

print(f"Num of people : {len(result)}")
# print(result_filmo[:10])


Now : 300
Now : 600
Now : 900
Now : 1200
Now : 1500
Now : 1800
Now : 2100
Now : 2400
Now : 2700
Now : 3000
Now : 3300
Now : 3600
Now : 3900
Now : 4200
Now : 4500
Now : 4800
Now : 5100
Now : 5400
Now : 5700
Now : 6000
Now : 6300
Now : 6600
Now : 6900
Now : 7200
Now : 7500
Now : 7800
Now : 8100
Now : 8400
Now : 8700
Now : 9000
Now : 9300
Now : 9600
Now : 9900
Now : 10200
Now : 10500
Now : 10800
Now : 11100
Now : 11400
Now : 11700
Now : 12000
Now : 12300
Now : 12600
Now : 12900
Now : 13200
Now : 13500
Now : 13800
Now : 14100
Now : 14400
Now : 14700
Num of people : 15000


## Pickle, Excel 파일로 저장하기


In [16]:
with open("people_info.pickle", 'wb') as bf:
    pickle.dump(result, bf)

df = pd.DataFrame.from_dict(result).T
with pd.ExcelWriter("./excel/people_info.xlsx", mode='w') as W:
    df.to_excel(W)

with open("filmo_collection.pickle",'wb') as bf :
    pickle.dump(result_filmo, bf)